<a href="https://colab.research.google.com/github/LimGuenTaek/Object_Detection/blob/master/FPEF_practice_and_visualization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 여기 cell 에서 image 하나 받아오고 , feature map 뽑아본뒤 visualization 해보고 기존 Feature 와 fused Feature 비교 해보기

import torch
import torch.nn as nn

feat4_3=torch.rand(1,512,38,38)
feat7=torch.rand(1,1024,19,19)
feat8_2=torch.rand(1,512,10,10)
feat9_2=torch.rand(1,256,5,5)
feat10_2=torch.rand(1,256,3,3)
feat11_2=torch.rand(1,256,1,1)

conv4_3_1=nn.Conv2d(512,256,1,stride=1)
conv4_3_3=nn.Conv2d(512,512,3,stride=1,padding=1)

conv7_1=nn.Conv2d(1024,256,1,stride=1)
conv7_3=nn.Conv2d(512,1024,3,stride=1,padding=1)

conv8_2_1=nn.Conv2d(512,256,1,stride=1)
conv8_2_3=nn.Conv2d(512,512,3,stride=1,padding=1)

conv9_2_1=nn.Conv2d(256,256,1,stride=1)
conv9_2_3=nn.Conv2d(512,256,3,stride=1,padding=1)

conv10_2_1=nn.Conv2d(256,256,1,stride=1)
conv10_2_3=nn.Conv2d(512,256,3,stride=1,padding=1)

Up7=nn.Upsample(scale_factor=2,mode="nearest")
Up8_2=nn.Upsample(scale_factor=1.9,mode="nearest")
Up9_2=nn.Upsample(scale_factor=2,mode="nearest")
Up10_2=nn.Upsample(scale_factor=5/3,mode="nearest")
Up11_2=nn.Upsample(scale_factor=3,mode="nearest")

# First step : applying 1 by 1 Conv for same channel 256
feat4_3_1=conv4_3_1(feat4_3)
feat7_1=conv7_1(feat7)
feat8_2_1=conv8_2_1(feat8_2)
feat9_2_1=conv9_2_1(feat9_2)
feat10_2_1=conv10_2_1(feat10_2)
feat11_2_1=feat11_2

# Second step : Upsampling before first fusion

Up_feat7=Up7(feat7_1)
Up_feat8_2=Up8_2(feat8_2_1)
Up_feat9_2=Up9_2(feat9_2_1)
Up_feat10_2=Up10_2(feat10_2_1)
Up_feat11_2=Up11_2(feat11_2_1)

# Third step : first fusion , element wise addition

fused_feat4_3=feat4_3_1+Up_feat7
fused_feat7=feat7_1+Up_feat8_2
fused_feat8_2=feat8_2_1+Up_feat9_2
fused_feat9_2=feat9_2_1+Up_feat10_2
fused_feat10_2=feat10_2_1+Up_feat11_2

# Forth step : second fusion , torch.cat(,dim=1) + conv2d(512,N,kernel_size=3,stride=1,padding=1)

feat4_3=conv4_3_3(torch.cat([feat4_3_1,fused_feat4_3],dim=1))
feat7=conv7_3(torch.cat([feat7_1,fused_feat7],dim=1))
feat8_2=conv8_2_3(torch.cat([feat8_2_1,fused_feat8_2],dim=1))
feat9_2=conv9_2_3(torch.cat([feat9_2_1,fused_feat9_2],dim=1))
feat10_2=conv10_2_3(torch.cat([feat10_2_1,fused_feat10_2],dim=1))

# And now return the feature to the prediction layer
print(feat4_3.shape) 
print(feat7.shape)   
print(feat8_2.shape)
print(feat9_2.shape)
print(feat10_2.shape)
print(feat11_2.shape)